# 作成したモデルのdeployとserving

０１_image_classificationで作ったモデルを今度はAPIとして使用できるようにします。

google ai-platformでは作成したmodelからendpointの作成を行う機能があるため、それを使ってみます。  
endpointの作成は2つの処理にわかれていて、modelの作成とversionの作成をおこなう必要があります。

In [ ]:
%%bash

USER=kojo
BUCKET=mixi-ml-handson-2020
VERSION=001

gcloud ai-platform models create mfashion_${USER} --regions asia-northeast1
gcloud ai-platform versions create mfashion_${USER}_${VERSION} \
  --model mfashion_${USER} \
  --origin gs://${BUCKET}/${USER}/${VERSION} \
  --runtime-version 2.1 \
  --python-version 3.7


上記コードの実行には数分の時間がかかります。

versionの作成が完了したら、このendpointにリクエストを投げてみましょう

リクエストは同一のproject内からを想定し、google-api-clientを使用します。

In [ ]:
!pip install google-api-python-client 

In [1]:
import googleapiclient.discovery

def predict_json(project, model, instances, version=None):

    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [3]:
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

(X_train_orig, y_train_orig), (X_test_orig, y_test_orig) = fashion_mnist.load_data()
requests=X_test_orig[0:8]
requests = np.expand_dims(requests, -1)
requests = requests / 255.0

predict_json("hr-mixi","mfashion_kojo", requests.tolist())


[{'output_1': [1.4490038045166442e-15,
   9.436579660145386e-17,
   1.3279680056066303e-14,
   1.3488622283870256e-18,
   4.852333058673482e-16,
   2.014125310978443e-09,
   3.877469075393115e-15,
   1.462386212125466e-08,
   2.452447579409878e-16,
   1.0]},
 {'output_1': [1.315963893411265e-09,
   3.667789973592804e-18,
   0.9999997615814209,
   3.8667758773570926e-14,
   1.6533456914658018e-07,
   2.5451908008556243e-19,
   1.4484761834410165e-07,
   7.995907766836618e-22,
   2.0016854927057404e-13,
   7.357056166676294e-20]},
 {'output_1': [1.0093023015663241e-13,
   1.0,
   7.563521145503171e-18,
   5.850782504963556e-14,
   1.190473043630397e-15,
   8.223065024923472e-19,
   3.110155226892716e-14,
   1.8334555212465892e-20,
   3.4673148112066975e-17,
   1.5001764048055636e-22]},
 {'output_1': [1.512368355549576e-12,
   1.0,
   3.634559084361895e-14,
   3.2286942563963805e-10,
   2.575585578146189e-12,
   5.374304618682973e-15,
   2.0595088134900408e-11,
   2.9912952650417855e-16,
